# 課題1：

以下に示すコードでは、cols[0]は正解極性、cols[1]は予測極性、dols[2]はコメントの要素になります。24行目から cols[0], cols[1]を参照しそれぞれのpositiveとnegativeをカウントしています。<br>
課題1では、week10で学んだ、TP, FP, FN, TNを求めます。<br>

4行目からの`def 関数名(引数)`は評価値を算出する処理を関数として定義しています。関数を使う場合は、`関数名(引数)`として使えます。（他の言語と同様ですね）<br>
36〜39行目に各評価指標の式を追加し、結果表示させ、分類精度を確認しましょう。

In [3]:
fname_result = 'result.txt' #sentence polarity dataset v1.0極性分類結果のファイル

#評価値を算出する関数 score(fname)を定義する
def score(fname):
    '''結果ファイルからスコア算出
    結果ファイルを読み込んで、正解率、適合率、再現率、F1スコアを返す
    戻り値：
    正解率,適合率,再現率,F1スコア
    '''
    # 結果を読み込んで集計する。各変数の初期値を代入する。
    TP = 0      # True-Positive     予想が+1、正解も+1
    FP = 0      # False-Positive    予想が+1、正解は-1
    FN = 0      # False-Negative    予想が-1、正解は+1
    TN = 0      # True-Negative     予想が-1、正解も-1

    with open(fname) as data_file: #ファイルをopenし，data_fileとする
        for line in data_file:
            cols = line.split('\t')    # "-1  -1  0.7992886809287588"の並びのデータをtabで分割しリスト化
#            print(cols)   #colsの内容を表示

            if len(cols) < 3:
                continue

            if cols[0] == '+1':         # 0番目：正解極性
                if cols[1] == '+1':     # 1番目：予想極性
                    TP += 1
                else:
                    FN += 1
            else:
                if cols[1] == '+1':
                    FP += 1
                else:
                    TN += 1

    # 各指標の算出式:ここに各指標の式をTP, FN, FP, TNを用い入力します。演算子は一般的な書式　＋、ー、＊、＼　優先する計算は（）を使えます。
    accuracy = (TP + TN) / (TP + FN + FP + TN)    # 正解率
    precision = TP / (TP+FP)     # 適合率
    recall = TP/(TP+FN)    # 再現率
    f1 = (2*precision*recall) / (recall + precision)   # F1スコア

    return accuracy, precision, recall, f1
#def score(fname)の終わり

# スコア算出 定義したscoreにデータを渡し，　return の返す値を左から順に各変数に格納する。
accuracy, precision, recall, f1 = score(fname_result)
print('正解率　\t{}\n適合率　\t{}\n再現率　\t{}\nF1スコア　\t{}'.format( accuracy, precision, recall, f1))


正解率　	0.7477485928705441
適合率　	0.7476092255765986
再現率　	0.748030018761726
F1スコア　	0.7478195629747725


# 課題2：

まず初めに、データになる頻度ベクトルを生成します。以下の手順でコード、コメントおよび式を確認しながら課題を進めていきましょう。`print`文の前の`#`コメント記号は適宜、削除/追加しながら進めてください。<br>

1. 23行目　corpusの頻度ベクトルの確認

### ※コードの行番号は、[ESC]+l もしくは操作タブの「表示」->行番号をトグル で表示/非表示ができます。

In [4]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import normalize #データの正規化などが可能な
import numpy as np #数値計算用ライブラリpnumpyをnpと名前をつけて以下で利用
import pandas as pd #データ解析用ライブラリpandasをpdと名前をつけて以下で利用

#以下の4文のリストを文書コーパスとして使う．
corpus = [
    'This is the first document.',
    'This document is the second document.',
    'And this is the third one.',
    'Is this the first document?'
]

smooth_idf = True
norm_idf = True

# 文章ごとの頻度ベクトルを作る
wc = CountVectorizer()
x = wc.fit_transform(corpus)
wcX = np.array(x.toarray())

print('4文コーパス: \n', corpus, '\n') #corpusを表示
print('頻度ベクトル: \n', wcX,'\n')  #orpusの頻度ベクトルの確認

4文コーパス: 
 ['This is the first document.', 'This document is the second document.', 'And this is the third one.', 'Is this the first document?'] 

頻度ベクトル: 
 [[0 1 1 1 0 0 1 0 1]
 [0 2 0 1 0 1 1 0 1]
 [1 0 0 1 1 0 1 1 1]
 [0 1 1 1 0 0 1 0 1]] 



# 課題3：


In [6]:
#ここまで実行したコードは有効なので，続きとして実行していきます．

# term frequency(TF) 単語の頻出度を求める:
N = wcX.shape[0]

tf = np.array([wcX[i, :] / np.sum(wcX, axis=1)[i] for i in range(N)])

print('4文コーパス: \n', corpus, '\n') #corpusを表示
print('全文書数： ', N, '\n') #全文書数の確認


print('wcX[i, :] \n', [wcX[i, :]for i in range(N)], '\n') #wcX[i, :]の確認
print('words_freq \n', [np.sum(wcX, axis=1)[i]for i in range(N)], '\n')#各文の単語の出現頻度(単語数)の確認
print('tf \n', tf, '\n')

# inverse documents frequency(IDF) 逆文書頻度を求める,  df:単語Xを含む文書数 :
df = np.count_nonzero(wcX, axis=0) 
print('df: ', df, '\n') #df の確認

idf = np.log((1 + N) / (1 + df) )+ 1  if smooth_idf else np.log( N / df )  #全文書数と単語Xを含む文書数に1を足しているのは，滑らかな値にするため
print('idf :', idf, '\n')

#TF-IDFを求め、正規化(normalize)する．
tfidf = normalize(tf*idf) if norm_idf else tf*idf

#tfidfを格納するデータフレーム(columns:列ラベル，index:行ラベル)形式にする. df_tifidf の dfはデータフレームの意味で付けました．
df_tfidf = pd.DataFrame(tfidf, columns=wc.get_feature_names())

print(df_tfidf)

4文コーパス: 
 ['This is the first document.', 'This document is the second document.', 'And this is the third one.', 'Is this the first document?'] 

全文書数：  4 

wcX[i, :] 
 [array([0, 1, 1, 1, 0, 0, 1, 0, 1]), array([0, 2, 0, 1, 0, 1, 1, 0, 1]), array([1, 0, 0, 1, 1, 0, 1, 1, 1]), array([0, 1, 1, 1, 0, 0, 1, 0, 1])] 

words_freq 
 [5, 6, 6, 5] 

tf 
 [[0.         0.2        0.2        0.2        0.         0.
  0.2        0.         0.2       ]
 [0.         0.33333333 0.         0.16666667 0.         0.16666667
  0.16666667 0.         0.16666667]
 [0.16666667 0.         0.         0.16666667 0.16666667 0.
  0.16666667 0.16666667 0.16666667]
 [0.         0.2        0.2        0.2        0.         0.
  0.2        0.         0.2       ]] 

df:  [1 3 2 4 1 1 4 1 4] 

idf : [1.91629073 1.22314355 1.51082562 1.         1.91629073 1.91629073
 1.         1.91629073 1.        ] 

        and  document     first        is       one    second       the  \
0  0.000000  0.469791  0.580286  0.384085  0

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [14]:
# print(df_tfidf['and']) # access based on column 
# print(df_tfidf.iloc[1]) # access based on row index

def getCharacteristicWord():
  result = [[] for i in range(len(df_tfidf["and"]))]
  # result[1].append("kkk")

  for column in df_tfidf.columns:
    for i in range(len(df_tfidf[column])):
      if (df_tfidf[column].iloc[i] > 0.45):
        result[i].append(column)
        # print(i, column)

  for i in range(len(result)):
    print("文書",i,result[i])

In [13]:
getCharacteristicWord()

文書 0 ['first']
文書 1 ['document', 'second']
文書 2 ['and', 'one', 'third']
文書 3 ['first']


文書ごとに各単語の特徴的である重要度が計算できました。<br>
出力結果を確認し、文書0~3の特徴のある単語($tf \cdot idf > 0.45$とした場合)を答えてください。複数ある場合は、カンマで区切ること。<br>
- 文書0: document, first
- 文書1: document, second
- 文書2: and, one, third
- 文書3: document, first

考察： 閾値を0.45としましたが，各文書の特徴的な単語になっているようでしょうか。   
出力結果を見て，適切な閾値を決める，または，上位何位までを特徴のある単語とするなどの方法で特徴ある単語として自動的に決めることが可能になります。

考察： 文章の特徴的な単語となる単語の最小値として 0.45 を使用すると、実際には文章の特徴を示さない単語がいくつか得られます。 <br>
条件として値を変更することで、結果はより正確になります。

# 課題4：

1. corpusは方法1と同じリストを利用します。ライブラリを読み込んだ後に追加してください。
2. corpusを追加する前の，6行目の `x`　に次の式を設定します。<br>
- TfidfVectorizer()でtf-idfを求める関数は `fit_transform()` が用意されています。
- 5行目で fidfVectorizer()のインスタンスを `tfidf`に設定します。
- 使い方は、`tfidf`の後に ドット`.`　で`fit_transform()`を指定するだけです。() には文書(ここではcorpus)を指定します。
3. 実行し、結果が方法1とほぼ同様になるか確認しましょう。

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

#TfidfVectorizerに用意されている .fit_transform を用いtf-idfを算出する。
tfidf = TfidfVectorizer()
x = tfidf.fit_transform(corpus);

#tfidfの結果をみるために、データフレームに変換します．pandasのimportが必要
df_tfidf = pd.DataFrame(x.toarray(), columns=tfidf.get_feature_names())

print('TF・IDF:')
print(df_tfidf)
print('\n')

TF・IDF:
        and  document     first        is       one    second       the  \
0  0.000000  0.469791  0.580286  0.384085  0.000000  0.000000  0.384085   
1  0.000000  0.687624  0.000000  0.281089  0.000000  0.538648  0.281089   
2  0.511849  0.000000  0.000000  0.267104  0.511849  0.000000  0.267104   
3  0.000000  0.469791  0.580286  0.384085  0.000000  0.000000  0.384085   

      third      this  
0  0.000000  0.384085  
1  0.000000  0.281089  
2  0.511849  0.267104  
3  0.000000  0.384085  




/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


TfidfVectorizer()を用いることで，単語のtf-idfを簡単に求めることができました。<br>
考察： 方法1と方法2の結果を比較するとどうですか？

考察：方法１と同じ結果が出ました。